In [3]:
line = '192.168.56.1 - - [30/Aug/2018:13:09:09 -0400] "GET /index.php?=PHPE9568F35-D428-11d2-A769-00AA001ACF42 HTTP/1.1" 200 2146 "http://192.168.56.101/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36"'

In [4]:
line

'192.168.56.1 - - [30/Aug/2018:13:09:09 -0400] "GET /index.php?=PHPE9568F35-D428-11d2-A769-00AA001ACF42 HTTP/1.1" 200 2146 "http://192.168.56.101/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36"'

In [22]:
from collections import namedtuple
import datetime
Request = namedtuple('Request',['method','url','version'])
Mapitem = namedtuple('Mapitem',['name','convert'])

mapping = [
    Mapitem('remote',lambda x:x),
    Mapitem('',None),
    Mapitem('',None),
    Mapitem('time',lambda x:datetime.datetime.strptime(x, "%d/%b/%Y:%H:%M:%S %z")),
    Mapitem('request',lambda x:Request(*x.split())),
    Mapitem('status',int),
    Mapitem('length',int),
    Mapitem('address',lambda x:x),
    Mapitem('ua',lambda x:x)
]

def extract(line):
    tmp = []
    ret = []
#    names=['remote','','','time','request','status','length','address','ua']
    split = True
    for c in line:
        if c == "[":
            split = False
            continue
        if c == "]":
            split = True
            continue
        if c == '"':
            split = not split
            continue
        if c ==" " and split:
            ret.append(''.join(tmp))
            tmp.clear()
        else:
            tmp.append(c)
    ret.append(''.join(tmp))
#    reasult = dict(zip(names,ret))
#    reasult.pop('')
    reasult = dict()
    for k,item in enumerate(mapping):
        if item.name:
            reasult[item.name] = item.convert(ret[k]) 
    return reasult

In [23]:
extract(line)

{'address': 'http://192.168.56.101/',
 'length': 2146,
 'remote': '192.168.56.1',
 'request': Request(method='GET', url='/index.php?=PHPE9568F35-D428-11d2-A769-00AA001ACF42', version='HTTP/1.1'),
 'status': 200,
 'time': datetime.datetime(2018, 8, 30, 13, 9, 9, tzinfo=datetime.timezone(datetime.timedelta(-1, 72000))),
 'ua': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'}

## 以上的程序对数据的格式要求很高

#正则表达式,文本处理的强大工具
#一个强大的工具,但也有多种实现方式
#BRE 基础正则表达式
#ERE 扩展正则表达式
#PCRE 以及PCRE的各种方言: perl 兼容正则表达式,功能最强
#python是sre,是pcre的一个子集

In [25]:
line

'192.168.56.1 - - [30/Aug/2018:13:09:09 -0400] "GET /index.php?=PHPE9568F35-D428-11d2-A769-00AA001ACF42 HTTP/1.1" 200 2146 "http://192.168.56.101/" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36"'

In [26]:
import re

In [44]:
regex = r'(?P<remote>\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}) - - \[(?P<time>.*)\] "(?P<method>.*)" (?P<status>\d+) (?P<length>\d+) "(?P<adress>.*)" "(?P<ua>.*)"'

In [67]:
Request = namedtuple('Request',['method','url','version'])
mapping = {
    'time': lambda x:datetime.datetime.strptime(x, "%d/%b/%Y:%H:%M:%S %z"),
    'request': lambda x:Request(*x.split()),
    'status': int,
    'length': int,
}
m = re.match(regex,line)#
if m:
    ret = m.groupdict()
    result = {}
    for k,v in ret.items():
        result[k] = mapping.get(k, lambda x:x)(v)
    print(result)

{'adress': 'http://192.168.56.101/', 'status': 200, 'method': 'GET /index.php?=PHPE9568F35-D428-11d2-A769-00AA001ACF42 HTTP/1.1', 'remote': '192.168.56.1', 'length': 2146, 'time': datetime.datetime(2018, 8, 30, 13, 9, 9, tzinfo=datetime.timezone(datetime.timedelta(-1, 72000))), 'ua': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36'}


In [2]:
#the difference between re.match and re.search 
import re
text = 'abcdef'
re.match('b',text) #match 从头匹配

In [3]:
re.search('b',text) #search可以从任意位置匹配

<_sre.SRE_Match object; span=(1, 2), match='b'>

## 滑动窗口的实现

In [1]:
import time
import datetime
from collections import namedtuple
import queue
import random
import time

Handler = namedtuple('Handler',['handle','notify'])
q = queue.Queue()

def source():
    while True:
        yield({'time':datetime.datetime.now(),'value':random.randint(0,100)})
        time.sleep(1)

def handler(items):
    values = [x['value'] for x in items]
    print(sum(values) / len(values))        

def window(source, handler:Handler,interval:int,width:int):
    store = []
    start = datetime.datetime.now()
    while True:
        data = next(source)
        current = datetime.datetime.now()
        if data:
            store.append(data)
            current = data['time']
        if (current - start).total_seconds() >= interval:
            start = current
            handler(store)
            dt = current - datetime.timedelta(seconds=width)
            store = [x for x in store if x['time'] > dt]
            print(store)

In [2]:
s = source()

In [6]:
for i in `ls *.m3u8`; 
do
filename=`echo "$i" |cut -d"." -f1`
echo "ffmpeg -i $i -c copy -bsf:a aac_adtstoasc $filename.mp4;"
done

<generator object source at 0x7fcd4f7fd6d0>

In [ ]:
for i in range(11):
    print(q.get())

In [ ]:
q.get()

In [9]:
l = []